In [51]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import re


SRC = Path("leie_priser.xlsx")  
assert SRC.exists(), f"Fant ikke filen: {SRC.resolve()}"

In [52]:
#Read data from Excel file
data = pd.read_excel(SRC, sheet_name=0)

In [53]:
#Finding columns containing the years 2012-2024
år_kolonner = [
    kol for kol in data.columns
    if re.fullmatch(r"\d{4}", str(kol)) and 2012 <= int(kol) <= 2024
]
if not år_kolonner:
    raise ValueError("Fant ingen årstallskolonner (2012-2024) i Excel-filen!")
#Make data into long format
long_df = data.melt(
    id_vars=["Område", "Størrelse"],
    value_vars=år_kolonner,
    var_name="år",
    value_name="leiepris"
)
#Transform years to int and leiepris to numeric, drop NaNs
long_df["år"] = long_df["år"].astype(int)
long_df["leiepris"] = pd.to_numeric(long_df["leiepris"], errors="coerce")
long_df = long_df.dropna(subset=["leiepris"])


def split_størrelse_column(df, col_name="størrelse"):
    # Extract first value as integer into a column called rom
    df["rom"] = df[col_name].str.extract(r'(\d+)-roms').astype(int)
    
    # Extract size in square meters into a column called størrelse_m2
    df["størrelse_m2"] = df[col_name].str.extract(r'(\d+)\s*kvm').astype(int)
    
    return df

long_df = split_størrelse_column(long_df, col_name="Størrelse")

In [54]:
# Features og target
FEATURES_CATEGORICAL = ["Område"]
FEATURES_NUMERICAL = ["år", "rom", "størrelse_m2"]
TARGET = "leiepris"

X = long_df[FEATURES_CATEGORICAL + FEATURES_NUMERICAL]
y = long_df[TARGET]

# Filter training data on years before 2024, validation data on 2024
train_filter = long_df["år"] < 2024
val_filter   = long_df["år"] == 2024

X_train = X[train_filter]
y_train = y[train_filter]
X_val   = X[val_filter]
y_val   = y[val_filter]
X_copy = X.copy()

# One-hot encoding for categorical features
X_train = pd.get_dummies(X_train, columns=FEATURES_CATEGORICAL)
X_val   = pd.get_dummies(X_val, columns=FEATURES_CATEGORICAL)
X_copy = pd.get_dummies(X_copy, columns=FEATURES_CATEGORICAL)

X_val = X_val.reindex(columns=X_train.columns, fill_value=0)
# Align columns of validation set to training set
feature_columns = X_train.columns.tolist()
# Makes sure y is float
y_train = y_train.astype(float)
y_val   = y_val.astype(float)

In [55]:
import joblib

# Valide rom per område
rom_per_område = (long_df.dropna(subset=["rom"])
                  .groupby("Område")["rom"]
                  .unique().apply(lambda x: sorted(set(int(r) for r in x)))
                  .to_dict())

# Valid square meters per område og rom
m2_per_område_rom = (long_df.dropna(subset=["rom","størrelse_m2"])
                     .groupby(["Område","rom"])["størrelse_m2"]
                     .unique().apply(lambda x: sorted(set(int(v) for v in x)))
                     .to_dict())

joblib.dump(rom_per_område, "rom_per_område.pkl")
joblib.dump(m2_per_område_rom, "m2_per_område_rom.pkl")



['m2_per_område_rom.pkl']

In [56]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ridge = Ridge(alpha=10)  

# Train on 2012–2023
ridge.fit(X_train, y_train)

# Predict on 2024 validation data
pred_val_ridge = ridge.predict(X_val)

# Evaluation
mae_ridge = mean_absolute_error(y_val, pred_val_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_val, pred_val_ridge))
r2_ridge = r2_score(y_val, pred_val_ridge)

print("Ridge Regression Performance (Validation 2024)")
print(f"MAE  = {mae_ridge:,.0f} kr")
print(f"RMSE = {rmse_ridge:,.0f} kr")
print(f"R²   = {r2_ridge:.3f}")



Ridge Regression Performance (Validation 2024)
MAE  = 1,263 kr
RMSE = 1,737 kr
R²   = 0.878


In [57]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

num_cols = ["år", "rom", "størrelse_m2"]
cat_cols = [c for c in X_train.columns if c.startswith("Område_")]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols)
    ],
    remainder="passthrough", 
    verbose_feature_names_out=False
)

ridge = Ridge(random_state=42)

pipe = Pipeline([
    ("prep", preprocess),
    ("model", ridge),
])


param_grid = {
    "model__alpha": np.logspace(-2, 3, 20)  
    
}

tscv = TimeSeriesSplit(n_splits=5)

grid = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=tscv,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

best_ridge = grid.best_estimator_
print("Best alpha:", grid.best_params_["model__alpha"])
print("CV MAE (train years):", -grid.best_score_)

# Evaluate on 2024
pred_val_ridge = best_ridge.predict(X_val)
mae  = mean_absolute_error(y_val, pred_val_ridge)
rmse = np.sqrt(mean_squared_error(y_val, pred_val_ridge))
r2   = r2_score(y_val, pred_val_ridge)

print(f"MAE  = {mae:,.0f} kr")
print(f"RMSE = {rmse:,.0f} kr")
print(f"R²   = {r2:.3f}")


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best alpha: 0.01
CV MAE (train years): 882.7245719202386
MAE  = 1,250 kr
RMSE = 1,708 kr
R²   = 0.882


In [58]:
# Build evaluation DataFrame for 2024
val_results = X_val.copy()
val_results["Område"] = long_df.loc[val_filter, "Område"].values 
val_results["y_true"] = y_val.values
val_results["y_pred"] = pred_val_ridge
val_results["error"] = val_results["y_pred"] - val_results["y_true"]
val_results["abs_error"] = val_results["error"].abs()

val_results.head()



,år,rom,størrelse_m2,Område_Akershus - nærliggende Oslo kommuner,Område_Bergen - Bergenhus,Område_Bergen - øvrige bydeler,Område_Kristiansand,"Område_Oslo - Grunerløkka, Gamle Oslo, Sagene, Nordre Aker og Vestre Aker","Område_Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen","Område_Oslo - Søndre Nordstrand, Grorud, Stovner og Alna",...,Område_Tettsteder med 2 000 - 19 999 innbyggere,Område_Tromsø,Område_Trondheim - Lerkendal og Heimdal,"Område_Trondheim - Midtbyen, Østbyen og Nedre Elvehavn",Område_Utkant Akershus,Område,y_true,y_pred,error,abs_error
4800,2024,1,15,False,False,False,False,False,True,False,...,False,False,False,False,False,"Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen",10300.0,12114.298895,1814.298895,1814.298895
4801,2024,1,20,False,False,False,False,False,True,False,...,False,False,False,False,False,"Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen",11100.0,12317.585702,1217.585702,1217.585702
4802,2024,1,30,False,False,False,False,False,True,False,...,False,False,False,False,False,"Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen",12400.0,12724.159318,324.159318,324.159318
4803,2024,1,40,False,False,False,False,False,True,False,...,False,False,False,False,False,"Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen",13500.0,13130.732933,-369.267067,369.267067
4804,2024,1,50,False,False,False,False,False,True,False,...,False,False,False,False,False,"Oslo - Sentrum, Frogner, Ullern og St.Hanshaugen",14300.0,13537.306548,-762.693452,762.693452


In [59]:
mae_per_area = (
    val_results.groupby("Område")["abs_error"]
    .mean()
    .sort_values(ascending=False)
)

In [60]:


joblib.dump(best_ridge, "ridge_model.pkl")
joblib.dump(X_train.columns.tolist(), "feature_columns.pkl")
joblib.dump(list(long_df["Område"].unique()), "områder.pkl")

rom_per_område = (
    long_df.groupby("Område")["rom"]
    .unique()
    .apply(lambda x: sorted(set(int(r) for r in x)))
    .to_dict()
)
m2_per_område_rom = (
    long_df.groupby(["Område", "rom"])["størrelse_m2"]
    .unique()
    .apply(lambda x: sorted(set(int(v) for v in x)))
    .to_dict()
)
joblib.dump(rom_per_område, "rom_per_område.pkl")
joblib.dump(m2_per_område_rom, "m2_per_område_rom.pkl")




['m2_per_område_rom.pkl']

In [61]:
df_23_24 = long_df[long_df["år"].isin([2023, 2024])].copy()

area_year_avg = (
    df_23_24.groupby(["Område", "år"])["leiepris"]
    .mean()
    .unstack("år")
)

national_growth = (
    (df_23_24[df_23_24["år"]==2024]["leiepris"].mean() -
     df_23_24[df_23_24["år"]==2023]["leiepris"].mean()) /
     df_23_24[df_23_24["år"]==2023]["leiepris"].mean()
)

area_growth = ((area_year_avg[2024] - area_year_avg[2023]) / area_year_avg[2023]).replace([np.inf, -np.inf], np.nan)
area_growth = area_growth.fillna(national_growth)

joblib.dump(area_growth.to_dict(), "area_growth.pkl")
joblib.dump(float(national_growth), "national_growth.pkl")






['national_growth.pkl']